In [1]:
import json
import pandas as pd
import random
import numpy as np


In [2]:
from biological_fuzzy_logic_networks.DREAM import DREAMBioFuzzNet

import torch
import numpy as np
import pandas as pd
import json
import click
import os
import pickle
from sklearn.preprocessing import MinMaxScaler


def load_and_prepare_data(
    data_dir,
    noise_sd,
    add_noise_to_input_train,
    add_noise_to_y_train,
    add_noise_to_input_test,
    add_noise_to_y_test,
):
    train_true_df = pd.read_csv(f"{data_dir}/train_true_df.csv")
    train_input_df = pd.read_csv(f"{data_dir}/train_input_df.csv")
    test_true_df = pd.read_csv(f"{data_dir}/test_true_df.csv")
    test_input_df = pd.read_csv(f"{data_dir}/test_input_df.csv")

    if add_noise_to_y_train:
        # Add noise to training data (and test data?)
        train_input_noise = np.random.normal(0, noise_sd, train_input_df.shape)
        train_input_df = train_input_df + train_input_noise

    if add_noise_to_y_train:
        # Add noise to training data (and test data?)
        train_noise = np.random.normal(0, noise_sd, train_true_df.shape)
        train_true_df = train_true_df + train_noise

    if add_noise_to_input_test:
        # Add noise to test data
        input_noise = np.random.normal(0, noise_sd, test_input_df.shape)
        test_input_df = test_input_df + input_noise

    if add_noise_to_y_test:
        test_noise = np.random.normal(0, noise_sd, test_true_df.shape)
        test_true_df = test_true_df + test_noise

    return train_true_df, train_input_df, test_true_df, test_input_df


def run_train_with_noise(
    pkn_path,
    data_dir,
    train_frac: float = 0.7,
    noise_sd: int = 1,
    add_noise_to_input_train: bool = False,
    add_noise_to_y_train: bool = False,
    add_noise_to_input_test: bool = True,
    add_noise_to_y_test: bool = False,
    BFN_training_params: dict = {
        "epochs": 100,
        "batch_size": 500,
        "learning_rate": 0.001,
        "tensors_to_cuda": True,
    },
    **extras,
):

    student_network = DREAMBioFuzzNet.DREAMBioFuzzNet.build_DREAMBioFuzzNet_from_file(
        pkn_path
    )
    untrained_network = DREAMBioFuzzNet.DREAMBioFuzzNet.build_DREAMBioFuzzNet_from_file(
        pkn_path
    )

    # Get data with/without noise
    train_true_df, train_input_df, test_true_df, test_input_df = load_and_prepare_data(
        data_dir=data_dir,
        add_noise_to_input_train=add_noise_to_input_train,
        add_noise_to_y_train=add_noise_to_y_train,
        add_noise_to_input_test=add_noise_to_input_test,
        add_noise_to_y_test=add_noise_to_y_test,
        noise_sd=noise_sd,
    )
    test_size = len(test_true_df)

    # Train student on unperturbed training data
    # Split train data in training and validation data
    train = train_true_df.sample(frac=train_frac)
    val = train_true_df.drop(train.index, axis=0)
    train_size = len(train)
    val_size = len(val)

    # Same input as teacher:
    train_input = train_input_df.iloc[train.index, :]
    val_input = train_input_df.drop(train.index, axis=0)

    # Data should have root nodes and non-root nodes
    all_train = pd.concat([train, train_input], axis=1)
    all_val = pd.concat([val, val_input], axis=1)
    all_test = pd.concat([test_true_df, test_input_df], axis=1)

    # Train scaler on the training data
    scaler = MinMaxScaler()
    all_train = pd.DataFrame(
        scaler.fit_transform(all_train),
        columns=all_train.columns,
        index=all_train.index,
    )

    # Scale validation data and prepara tensors
    all_val = pd.DataFrame(
        scaler.transform(all_val),
        columns=all_val.columns,
        index=all_val.index,
    )

    all_test = pd.DataFrame(
        scaler.transform(all_test),
        columns=all_test.columns,
        index=all_test.index,
    )

    all_test[all_test>1] = 1
    all_val[all_val>1] = 1
    all_train[all_train>1] = 1
    
    val_input_dict = {c: torch.Tensor(np.array(all_val[c])) for c in val_input.columns}
    train_input_dict = {
        c: torch.Tensor(np.array(all_train[c])) for c in train_input.columns
    }
    train_dict = {c: torch.Tensor(np.array(all_train[c])) for c in all_train.columns}
    val_dict = {c: torch.Tensor(np.array(all_val[c])) for c in all_val.columns}

    # Data should have root nodes and non-root nodes
    val_dict.update(val_input_dict)

    # Inhibitors
    train_inhibitors = {c: torch.ones(len(all_train)) for c in train_dict.keys()}
    val_inhibitors = {c: torch.ones(len(all_val)) for c in val_dict.keys()}

    student_network.initialise_random_truth_and_output(
        train_size, to_cuda=BFN_training_params["tensors_to_cuda"]
    )
    losses, curr_best_val_loss, _ = student_network.conduct_optimisation(
        input=train_input_dict,
        ground_truth=train_dict,
        train_inhibitors=train_inhibitors,
        valid_ground_truth=val_dict,
        valid_input=val_input_dict,
        valid_inhibitors=val_inhibitors,
        **BFN_training_params,
    )

    test_ground_truth = {
        c: torch.Tensor(np.array(all_test[c])) for c in all_test.columns
    }

    no_inhibition_test = {k: torch.ones(test_size) for k in student_network.nodes}
    student_network.initialise_random_truth_and_output(
        test_size, to_cuda=BFN_training_params["tensors_to_cuda"]
    )
    student_network.set_network_ground_truth(
        test_ground_truth, to_cuda=BFN_training_params["tensors_to_cuda"]
    )
    
    student_network.sequential_update(
        student_network.root_nodes,
        inhibition=no_inhibition_test,
        to_cuda=BFN_training_params["tensors_to_cuda"],
    )
    with torch.no_grad():
        test_output = {
            k: v.cpu()
            for k, v in student_network.output_states.items()
            if k not in student_network.root_nodes
        }
        test_output_df = pd.DataFrame({k: v.numpy() for k, v in test_output.items()})

    # TEST student network without perturbation, random inputs
    student_network.initialise_random_truth_and_output(
        test_size, to_cuda=BFN_training_params["tensors_to_cuda"]
    )
    student_network.sequential_update(
        student_network.root_nodes,
        inhibition=no_inhibition_test,
        to_cuda=BFN_training_params["tensors_to_cuda"],
    )
    with torch.no_grad():
        test_random_output = {
            k: v.cpu()
            for k, v in student_network.output_states.items()
            if k not in student_network.root_nodes
        }
        test_random_output_df = pd.DataFrame(
            {k: v.numpy() for k, v in test_random_output.items()}
        )

    # UNTRAINED NETWORK without perturbation, same inputs
    untrained_network.initialise_random_truth_and_output(test_size)
    untrained_network.set_network_ground_truth(test_ground_truth)
    untrained_network.sequential_update(
        untrained_network.root_nodes, inhibition=no_inhibition_test
    )
    with torch.no_grad():
        gen_with_i_test = {
            k: v.cpu().numpy()
            for k, v in untrained_network.output_states.items()
            if k not in untrained_network.root_nodes
        }
        ut_test_with_i_df = pd.DataFrame(gen_with_i_test)

    # UNTRAINED NETWORK without perturbation, random inputs
    untrained_network.initialise_random_truth_and_output(test_size)
    untrained_network.sequential_update(
        untrained_network.root_nodes, inhibition=no_inhibition_test
    )
    with torch.no_grad():
        gen_test = {
            k: v.cpu().numpy()
            for k, v in untrained_network.output_states.items()
            if k not in untrained_network.root_nodes
        }
        ut_test_df = pd.DataFrame(gen_test)

    unpertubed_pred_data = pd.concat(
        [
            test_true_df,
            test_output_df,
            test_random_output_df,
            ut_test_with_i_df,
            ut_test_df,
        ],
        keys=[
            "teacher_true",
            "student_same_input",
            "student_random_input",
            "untrained_same_input",
            "untrained_random_input",
        ],
    )

    return (losses, unpertubed_pred_data, student_network.get_checkpoint(), scaler)

def minimal_train(pkn_path, BFN_training_params, all_train, all_val, all_test):
    
    all_test[all_test>1] = 1
    all_val[all_val>1] = 1
    all_train[all_train>1] = 1
            
    test_size = len(all_test)
    val_input_dict = {c: torch.Tensor(np.array(all_val[c])) for c in train_input_df.columns}
    train_input_dict = {
        c: torch.Tensor(np.array(all_train[c])) for c in train_input_df.columns
    }
    train_dict = {c: torch.Tensor(np.array(all_train[c])) for c in all_train.columns}
    val_dict = {c: torch.Tensor(np.array(all_val[c])) for c in all_val.columns}

    # Inhibitors
    train_inhibitors = {c: torch.ones(len(all_train)) for c in train_dict.keys()}
    val_inhibitors = {c: torch.ones(len(all_val)) for c in val_dict.keys()}

    # Test set
    test_ground_truth = {
        c: torch.Tensor(np.array(all_test[c])) for c in all_test.columns
    }

    no_inhibition_test = {k: torch.ones(len(all_test)) for k in train_dict.keys()}
    
    student_network = DREAMBioFuzzNet.DREAMBioFuzzNet.build_DREAMBioFuzzNet_from_file(
        pkn_path
    )
    untrained_network = DREAMBioFuzzNet.DREAMBioFuzzNet.build_DREAMBioFuzzNet_from_file(
        pkn_path
    )

    student_network.initialise_random_truth_and_output(
        len(all_train), to_cuda=BFN_training_params["tensors_to_cuda"]
    )
    
    losses, curr_best_val_loss, _ = student_network.conduct_optimisation(
        input=train_input_dict,
        ground_truth=train_dict,
        train_inhibitors=train_inhibitors,
        valid_ground_truth=val_dict,
        valid_input=val_input_dict,
        valid_inhibitors=val_inhibitors,
        **BFN_training_params,
    )

    with torch.no_grad():
            
        student_network.initialise_random_truth_and_output(
            test_size, to_cuda=BFN_training_params["tensors_to_cuda"]
        )
        student_network.set_network_ground_truth(
            test_ground_truth, to_cuda=BFN_training_params["tensors_to_cuda"]
        )

        student_network.sequential_update(
            student_network.root_nodes,
            inhibition=no_inhibition_test,
            to_cuda=BFN_training_params["tensors_to_cuda"],
        )
        test_output = {
            k: v.cpu()
            for k, v in student_network.output_states.items()
            if k not in student_network.root_nodes
        }
        test_output_df = pd.DataFrame({k: v.numpy() for k, v in test_output.items()})

    # TEST student network without perturbation, random inputs
    with torch.no_grad():
        student_network.initialise_random_truth_and_output(
        test_size, to_cuda=BFN_training_params["tensors_to_cuda"]
        )
        student_network.sequential_update(
            student_network.root_nodes,
            inhibition=no_inhibition_test,
            to_cuda=BFN_training_params["tensors_to_cuda"],
        )
        test_random_output = {
            k: v.cpu()
            for k, v in student_network.output_states.items()
            if k not in student_network.root_nodes
        }
        test_random_output_df = pd.DataFrame(
            {k: v.numpy() for k, v in test_random_output.items()}
        )

    # UNTRAINED NETWORK without perturbation, same inputs
    with torch.no_grad():
        untrained_network.initialise_random_truth_and_output(test_size)
        untrained_network.set_network_ground_truth(test_ground_truth)
        untrained_network.sequential_update(
            untrained_network.root_nodes, inhibition=no_inhibition_test
        )
        gen_with_i_test = {
            k: v.cpu().numpy()
            for k, v in untrained_network.output_states.items()
            if k not in untrained_network.root_nodes
        }
        ut_test_with_i_df = pd.DataFrame(gen_with_i_test)

    # UNTRAINED NETWORK without perturbation, random inputs
    with torch.no_grad():
        untrained_network.initialise_random_truth_and_output(test_size)
        untrained_network.sequential_update(
            untrained_network.root_nodes, inhibition=no_inhibition_test
        )
        gen_test = {
            k: v.cpu().numpy()
            for k, v in untrained_network.output_states.items()
            if k not in untrained_network.root_nodes
        }
        ut_test_df = pd.DataFrame(gen_test)

#     unpertubed_pred_data = pd.concat(
#         [
#             test_true_df,
#             test_output_df,
#             test_random_output_df,
#             ut_test_with_i_df,
#             ut_test_df,
#         ],
#         keys=[
#             "teacher_true",
#             "student_same_input",
#             "student_random_input",
#             "untrained_same_input",
#             "untrained_random_input",
#         ],
#     )

#     return (losses, unpertubed_pred_data, student_network.get_checkpoint())


/dccstor/ipc1/.conda/envs/biofuzznet2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open("/dccstor/ipc1/CAR/BFN/Model/Noise/Configs/noise4_tri_try_tei_noise_config.json", "rb") as f:
    config = json.load(f)
config["data_dir"] = config["data_dir"] + '4/'
config["BFN_training_params"]["batch_size"] = 1000
config["BFN_training_params"]["epochs"] = 100
config["pkn_path"]="/dccstor/ipc1/CAR/BFN/LiverDREAM_PKN.sif"

In [4]:
# from sklearn.preprocessing import MinMaxScaler
# from biological_fuzzy_logic_networks.Synthetic_experiments.training_noise import load_and_prepare_data
# train_true_df, train_input_df, test_true_df, test_input_df = load_and_prepare_data("/dccstor/ipc1/CAR/BFN/Model/Noise/SimData1/", 
#                                                                                   add_noise_to_input_test=config["add_noise_to_input_test"],
#                                                                                   add_noise_to_input_train=config["add_noise_to_input_train"],
#                                                                                   add_noise_to_y_test=config["add_noise_to_y_test"],
#                                                                                   add_noise_to_y_train=config["add_noise_to_y_train"],
#                                                                                   noise_sd=config["noise_sd"])

# # Train student on unperturbed training data
# # Split train data in training and validation data
# train = train_true_df.sample(frac=config["train_frac"])
# val = train_true_df.drop(train.index, axis=0)
# train_size = len(train)
# val_size = len(val)

# # Same input as teacher:
# train_input = train_input_df.iloc[train.index, :]
# val_input = train_input_df.drop(train.index, axis=0)

# # Data should have root nodes and non-root nodes
# all_train = pd.concat([train, train_input], axis=1)
# all_val = pd.concat([val, val_input], axis=1)

# all_train.to_csv("/dccstor/ipc1/CAR/BFN/Model/Test/temp_all_train_before_pp.csv")
# all_train.to_csv("/dccstor/ipc1/CAR/BFN/Model/Test/temp_all_val_before_pp.csv")

# all_train[all_train == 0] = 1e-5
# all_val[all_val == 0] = 1e-5

# # Train scaler on the training data
# scaler = MinMaxScaler()
# all_train = pd.DataFrame(
#     scaler.fit_transform(all_train),
#     columns=all_train.columns,
#     index=all_train.index,
# )

# # Scale validation data and prepara tensors
# all_val = pd.DataFrame(
#     scaler.transform(all_val),
#     columns=all_val.columns,
#     index=all_val.index,
# )

# all_train.to_csv("/dccstor/ipc1/CAR/BFN/Model/Test/temp_all_train_after_pp.csv")
# all_train.to_csv("/dccstor/ipc1/CAR/BFN/Model/Test/temp_all_val_after_pp.csv")

# # TEST student without perturbation, same inputs
# all_test = pd.concat([test_true_df, test_input_df], axis=1)
# all_test.to_csv("/dccstor/ipc1/CAR/BFN/Model/Test/temp_all_test_before_pp.csv")
# all_test = pd.DataFrame(
#     scaler.transform(all_test),
#     columns=all_test.columns,
#     index=all_test.index,
# )

# all_test[all_test == 0] = 1e-5
# all_test.to_csv("/dccstor/ipc1/CAR/BFN/Model/Test/temp_all_test_after_pp.csv")

In [5]:
from biological_fuzzy_logic_networks.Synthetic_experiments.training_noise import run_train_with_noise


In [ ]:
for i in range(int(1e6)):
#     os.environ["PL_GLOBAL_SEED"] = str(0)
#     random.seed(0)
#     np.random.seed(0)
#     torch.use_deterministic_algorithms(True)
#     torch.manual_seed(0)
#     torch.cuda.manual_seed_all(0)
    (losses, unpertubed_pred_data, student_network, scaler) = run_train_with_noise(**config)


Loss=?.??e??:  21%|█████▉                      | 21/100 [00:20<01:16,  1.04it/s]


Early stopping. Epoch: 21 batch: 7


Loss=?.??e??:  21%|█████▉                      | 21/100 [00:21<01:19,  1.01s/it]


Early stopping. Epoch: 21 batch: 7


Loss=?.??e??:  21%|█████▉                      | 21/100 [00:19<01:13,  1.08it/s]


...

Loss=?.??e??:  21%|█████▉                      | 21/100 [00:19<01:12,  1.09it/s]


Early stopping. Epoch: 21 batch: 7


Loss=?.??e??:  21%|█████▉                      | 21/100 [00:19<01:12,  1.08it/s]


Early stopping. Epoch: 21 batch: 7


Loss=?.??e??:  15%|████▏                       | 15/100 [00:13<01:16,  1.11it/s]

In [ ]:
config["data_dir"] = config["data_dir"] + '4/'
config["BFN_training_params"]["batch_size"] = 10
config["BFN_training_params"]["epochs"] = 1
config["pkn_path"]="/dccstor/ipc1/CAR/BFN/LiverDREAM_PKN.sif"

In [ ]:
_, train_input_df, _, test_input_df = load_and_prepare_data("/dccstor/ipc1/CAR/BFN/Model/Noise/SimData1/", 
                                                                                  add_noise_to_input_test=config["add_noise_to_input_test"],
                                                                                  add_noise_to_input_train=config["add_noise_to_input_train"],
                                                                                  add_noise_to_y_test=config["add_noise_to_y_test"],
                                                                                  add_noise_to_y_train=config["add_noise_to_y_train"],
                                                                                  noise_sd=config["noise_sd"])

all_train = pd.read_csv("/dccstor/ipc1/CAR/BFN/Model/Test/temp_all_train_after_pp.csv", index_col=0)
all_val = pd.read_csv("/dccstor/ipc1/CAR/BFN/Model/Test/temp_all_val_after_pp.csv", index_col=0)
all_test = pd.read_csv("/dccstor/ipc1/CAR/BFN/Model/Test/temp_all_test_after_pp.csv", index_col=0)

before_pp_all_train = pd.read_csv("/dccstor/ipc1/CAR/BFN/Model/Test/temp_all_train_before_pp.csv", index_col=0)
before_pp_all_val = pd.read_csv("/dccstor/ipc1/CAR/BFN/Model/Test/temp_all_val_before_pp.csv", index_col=0)
before_pp_all_test = pd.read_csv("/dccstor/ipc1/CAR/BFN/Model/Test/temp_all_test_before_pp.csv", index_col=0)

In [ ]:
print("train: ", all_train.max().max(), "val: ", all_val.max().max(), "test: ", all_test.max().max())
print("train: ", all_train.min().min(), "val: ", all_val.min().min(), "test: ", all_test.min().min())

print("After PP")
print("train: ", before_pp_all_train.max().max(), "val: ", before_pp_all_val.max().max(), "test: ", before_pp_all_test.max().max())
print("train: ", before_pp_all_train.min().min(), "val: ", before_pp_all_val.min().min(), "test: ", before_pp_all_test.min().min())


In [ ]:
os.environ["PL_GLOBAL_SEED"] = str(0)
random.seed(0)
np.random.seed(0)
torch.use_deterministic_algorithms(True)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
try:
    minimal_train(config["pkn_path"], 
                  config['BFN_training_params'], 
                  all_train, 
                  all_val, 
                  all_test) # Print torch.seed()
except RuntimeError:
    print("Model stopped")

In [ ]:
from biological_fuzzy_logic_networks.Hill_function import HillTransferFunction

In [ ]:
hill = HillTransferFunction()
hill.K = torch.nn.Parameter(torch.Tensor([-0.9838]))
hill.n = torch.nn.Parameter(torch.Tensor([0.3227]))

In [ ]:
hill.forward(x=torch.Tensor([0.3553, 0.3177, 0.5697, 0.6115, 0.6100,train_input_dict["tgfa"].max().item(), 0.7479, 0.7486, 0.4753,
        0.4864]))

In [ ]:
train_input_dict = {
        c: torch.Tensor(np.array(all_train[c])) for c in train_input_df.columns
    }

In [ ]:
train_input_dict["tgfa"].max().item()

In [ ]:
### FINDING PARAMETER SEED WITH NAN GRADIENT

# for i in range(int(1e6)):
#     os.environ["PL_GLOBAL_SEED"] = str(0)
#     random.seed(0)
#     np.random.seed(0)
#     torch.use_deterministic_algorithms(True)
#     torch.manual_seed(0)
#     torch.cuda.manual_seed_all(0)
#     try:
#         minimal_train(config["pkn_path"], 
#                       config['BFN_training_params'], 
#                       all_train, 
#                       all_val, 
#                       all_test) # Print torch.seed()
#     except RuntimeError:
#         print("Model stopped at: ", i)